<a href="https://colab.research.google.com/github/HakureiPOI/Comp_Arch_Lab/blob/main/Arch_Lab1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/HakureiPOI/Comp_Arch_Lab.git

Cloning into 'Comp_Arch_Lab'...
remote: Enumerating objects: 36, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 36 (delta 13), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (36/36), 1.14 MiB | 3.51 MiB/s, done.
Resolving deltas: 100% (13/13), done.


---
### Practice 1

In [2]:
!cp Comp_Arch_Lab/lab1.tar.gz lab1.tar.gz

In [3]:
!tar -zxvf lab1.tar.gz

./lab1/
./lab1/src/
./lab1/src/lab1/
./lab1/src/lab1/test_gemm_kernel.cpp
./lab1/src/lab1/gemm.cpp
./lab1/src/lab1/CMakeLists.txt
./lab1/src/lab1/print_integer.S
./lab1/src/lab1/gemm_kernel.h
./lab1/src/lab1/gemm_kernel.S
./lab1/CMakeLists.txt
./lab1/cmake/
./lab1/cmake/env_params.cmake
./lab1/cmake/utils.cmake
./lab1/env/
./lab1/env/cpu_parallel_env.yaml
./lab1/env/cpu_parallel_env.dockerfile
./lab1/tools/
./lab1/tools/lmbench.tgz
./lab1/tools/10.251.137.194.0.log


In [4]:
!sudo apt update
!sudo apt install build-essential net-tools git vim cmake gdb make gfortran libnuma-dev libtirpc-dev

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [3,275 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,164 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security/ma

In [5]:
!cmake --version
!gcc --version
!gdb --version

cmake version 3.30.5

CMake suite maintained and supported by Kitware (kitware.com/cmake).
gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0
Copyright (C) 2021 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

GNU gdb (Ubuntu 12.1-0ubuntu1~22.04.2) 12.1
Copyright (C) 2022 Free Software Foundation, Inc.
License GPLv3+: GNU GPL version 3 or later <http://gnu.org/licenses/gpl.html>
This is free software: you are free to change and redistribute it.
There is NO WARRANTY, to the extent permitted by law.


In [6]:
!cp -f /content/Comp_Arch_Lab/lab1/print_integer.S /content/lab1/src/lab1/print_integer.S
!cat /content/lab1/src/lab1/print_integer.S

/**
 *  向标准输出打1个64位整数
 * */

.section .bss
//    .lcomm num, 8       // 存储64位整数
    .lcomm buffer, 21   // 20个数字 + 1个空字符 作为输出的缓冲区

.section .data
    newline: .byte 0xA      // 换行符

.section .text
    .globl _start

_start:
    // 初始化要打印的数字
    mov $1234567890123456789, %rax
//    mov %rax, num(%rip)

    // 将整数转换为字符串
//    mov num(%rip), %rax
    lea buffer+20(%rip), %rdi   // 将输出字符串的最后一个字符地址放入rdi存器
    movb $0, (%rdi)             // 给取后一个字符赋'\0'标志结束

convert_loop:                   // 将整数转换为字符串等待输出
    xor %rdx, %rdx
    mov $10, %rcx
    div %rcx                    // rdx = rax % 10, rax = rax / 10
    add $'0', %dl              // 计算对应的ascii码, rdx存器的低8位叫dl存器
    dec %rdi
    mov %dl, (%rdi)             // 结果写内存
    test %rax, %rax             // 测试rax是否为0
    jnz convert_loop

find_start:                     // 转换结束，跳过字符串前的所有0
    cmpb $'0', (%rdi)
    jne print_string
    inc %rdi
    jmp find_start

print_string:                   // 开始输出字符串
                                // 计算字

In [7]:
%cd /content/lab1

/content/lab1


In [8]:
!mkdir -p build
%cd build

/content/lab1/build


In [9]:
!cmake -DCMAKE_BUILD_TYPE=Debug ..
!cmake -B . -S ../ && cmake --build ./ --target lab1_print_integer

-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/gcc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/g++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- CMAKE_BUILD_TYPE is Debug
-- The ASM compiler identification is GNU
-- Found assembler: /usr/bin/gcc
-- Found Python3: /usr/local/bin/python (found version "3.10.12") found components: Interpreter
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD - Success
-- Found Threads: TRUE
-- Configuring done (7.5s)
-- Generating done (0.0s)
-- Build files have been written to: /content/lab1/build
-- CMAKE_BUILD_TYPE is Debug
-- Configuring done (0.7s)
-- Genera

In [10]:
%cd dist/bins/

/content/lab1/build/dist/bins


In [11]:
! ./lab1_print_integer

 123456789012345678


---
### Practice 2

In [12]:
%cd /content

/content


In [13]:
!cp -f /content/Comp_Arch_Lab/lab1/gemm_kernel.S /content/lab1/src/lab1/gemm_kernel.S
!cat /content/lab1/src/lab1/gemm_kernel.S

.text;
.p2align 2;
.global gemm_kernel;
.type gemm_kernel, %function;

// 以下是实现的实例代码
#define     MAT_C               %rdi
#define     MAT_A               %rsi
#define     MAT_B               %r14
#define     DIM_M               %rcx
#define     DIM_N               %r8
#define     DIM_K               %r9
#define     loop_m              %r10
#define     loop_k              %r11
#define     loop_n              %r12
#define     mat_elem_idx        %r13


.macro PUSHD                                        // 保存原通用寄存器值
    push %rax
    push %rbx
    push %rcx
    push %rdx
    push %rsi
    push %rdi
    push %rbp
    push %r8
    push %r9
    push %r10
    push %r11
    push %r12
    push %r13
    push %r14
    push %r15
.endm

.macro POPD                                        // 恢复原通用寄存器值
    pop %r15
    pop %r14
    pop %r13
    pop %r12
    pop %r11
    pop %r10
    pop %r9
    pop %r8
    pop %rbp
    pop %rdi
    pop %rsi
    pop %rdx
    pop %rcx
    pop %rbx
    pop %rax
.endm

.

In [14]:
%cd /content/lab1

/content/lab1


In [15]:
!mkdir -p build
%cd build

/content/lab1/build


In [16]:
!cmake -B . -S ../ && cmake --build ./ --target lab1_test_gemm_kernel.unittest

-- CMAKE_BUILD_TYPE is Debug
-- Configuring done (0.6s)
-- Generating done (0.0s)
-- Build files have been written to: /content/lab1/build
-- CMAKE_BUILD_TYPE is Debug
-- Configuring done (0.5s)
-- Generating done (0.0s)
-- Build files have been written to: /content/lab1/build
[ 14%] Building CXX object _deps/googletest-build/googletest/CMakeFiles/gtest.dir/src/gtest-all.cc.o
[ 28%] Linking CXX static library ../../../lib/libgtest.a
[ 28%] Built target gtest
[ 42%] Building CXX object _deps/googletest-build/googletest/CMakeFiles/gtest_main.dir/src/gtest_main.cc.o
[ 57%] Linking CXX static library ../../../lib/libgtest_main.a
[ 57%] Built target gtest_main
[ 71%] Building CXX object src/lab1/CMakeFiles/lab1_test_gemm_kernel.unittest.dir/test_gemm_kernel.cpp.o
[ 85%] Building ASM object src/lab1/CMakeFiles/lab1_test_gemm_kernel.unittest.dir/gemm_kernel.S.o
/content/lab1/src/lab1/gemm_kernel.S: Assembler messages:
/content/lab1/src/lab1/gemm_kernel.S:110: Warning: translating to `faddp %s

In [17]:
%cd dist/bins
! ./lab1_test_gemm_kernel.unittest --gtest_filter=gemm_kernel.test0

/content/lab1/build/dist/bins
Running main() from /content/lab1/build/_deps/googletest-src/googletest/src/gtest_main.cc
Note: Google Test filter = gemm_kernel.test0
[==========] Running 1 test from 1 test suite.
[----------] Global test environment set-up.
[----------] 1 test from gemm_kernel
[ RUN      ] gemm_kernel.test0


In [18]:
%cd /content/lab1

/content/lab1


In [19]:
!mkdir -p build
%cd build

/content/lab1/build


In [20]:
!cmake -B . -S ../ && cmake --build ./ --target lab1_gemm

-- CMAKE_BUILD_TYPE is Debug
-- Configuring done (0.5s)
-- Generating done (0.0s)
-- Build files have been written to: /content/lab1/build
[ 33%] Building CXX object src/lab1/CMakeFiles/lab1_gemm.dir/gemm.cpp.o
/content/lab1/src/lab1/gemm.cpp: In function ‘void random_matrix(int, int, float*)’:
/content/lab1/src/lab1/gemm.cpp:35:19: warning: empty parentheses were disambiguated as a function declaration []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wvexing-parse-Wvexing-parse]8;;]
   35 |     double drand48();
      |                   ^~
/content/lab1/src/lab1/gemm.cpp:35:19: note: remove parentheses to default-initialize a variable
   35 |     double drand48();
      |                   ^~
      |                   --
/content/lab1/src/lab1/gemm.cpp:35:19: note: or replace parentheses with braces to value-initialize a variable
[ 66%] Building ASM object src/lab1/CMakeFiles/lab1_gemm.dir/gemm_kernel.S.o
/content/lab1/src/lab1/gemm_kernel.S: Assembler messages:

In [21]:
%cd dist/bins/

/content/lab1/build/dist/bins


In [22]:
! ./lab1_gemm 256 256 256

GEMM performance info:
		M, K, N: 256, 256, 256
		Ops: 0.0335544


---
### Practice 3

In [23]:
!pwd

/content/lab1/build/dist/bins


In [24]:
!lscpu

Architecture:             x86_64
  CPU op-mode(s):         32-bit, 64-bit
  Address sizes:          46 bits physical, 48 bits virtual
  Byte Order:             Little Endian
CPU(s):                   2
  On-line CPU(s) list:    0,1
Vendor ID:                GenuineIntel
  Model name:             Intel(R) Xeon(R) CPU @ 2.20GHz
    CPU family:           6
    Model:                79
    Thread(s) per core:   2
    Core(s) per socket:   1
    Socket(s):            1
    Stepping:             0
    BogoMIPS:             4399.99
    Flags:                fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 cl
                          flush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc re
                          p_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3
                           fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand
                           hypervisor lahf_lm abm 3dnowprefetch i

In [30]:
!ls /sys/devices/system/cpu

cpu0  cpufreq  hotplug	 kernel_max  offline  possible	present  uevent
cpu1  cpuidle  isolated  modalias    online   power	smt	 vulnerabilities


In [31]:
!ls /sys/devices/system/cpu/cpu0/cache

index0	index1	index2	index3	uevent


In [25]:
%cd /sys/devices/system/cpu/cpu0/cache
%cd index0

/sys/devices/system/cpu/cpu0/cache
/sys/devices/system/cpu/cpu0/cache/index0


In [27]:
# 查看缓存行大小
!cat coherency_line_size
# 查看组数
!cat number_of_sets
# 查看way数
!cat ways_of_associativity

64
64
8


---
### Practice 4

In [34]:
!uname -r

6.1.85+


In [35]:
!apt search linux-tools

Sorting... Done
Full Text Search... Done
linux-tools-5.15.0-100/jammy-updates,jammy-security 5.15.0-100.110 amd64
  Linux kernel version specific tools for version 5.15.0-100

linux-tools-5.15.0-100-generic/jammy-updates,jammy-security 5.15.0-100.110 amd64
  Linux kernel version specific tools for version 5.15.0-100

linux-tools-5.15.0-100-lowlatency/jammy-updates,jammy-security 5.15.0-100.110 amd64
  Linux kernel version specific tools for version 5.15.0-100

linux-tools-5.15.0-1002-gke/jammy 5.15.0-1002.2 amd64
  Linux kernel version specific tools for version 5.15.0-1002

linux-tools-5.15.0-1002-ibm/jammy 5.15.0-1002.2 amd64
  Linux kernel version specific tools for version 5.15.0-1002

linux-tools-5.15.0-1002-oracle/jammy 5.15.0-1002.4 amd64
  Oracle Linux kernel version specific tools for version 5.15.0-1002

linux-tools-5.15.0-1003-azure/jammy 5.15.0-1003.4 amd64
  Linux kernel version specific tools for version 5.15.0-1003

linux-tools-5.15.0-1003-gcp/jammy 5.15.0-1003.6 amd64
 

In [38]:
!sudo apt install linux-tools-6.2.0-25-generic

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
linux-tools-6.2.0-25-generic is already the newest version (6.2.0-25.25~22.04.2).
0 upgraded, 0 newly installed, 0 to remove and 51 not upgraded.


In [39]:
!perf list


  You may need to install the following packages for this specific kernel:
    linux-tools-6.1.85+-6.1.85+
    linux-cloud-tools-6.1.85+-6.1.85+

  You may also want to install one of the following packages to keep up to date:
    linux-tools-6.1.85+
    linux-cloud-tools-6.1.85+
